
## Extraction of Features from Satellite Imagery
Using the images marked as valid per cluster, we pass them through the CNN and extract their feature vectors.  
Input: 
a. models/night28vggtrained_model.pt (saved models from 06_trainmodel notebook)
b. data/processed/nightimage_download_actual.csv

Output :
4096 features extracted from the images and stored in 
a. results/nepal_2018/cnnnighttrainvggcluster_feats.npy 
b. results/nepl_2018/cnnnighttrainvggcluster_order.pkl

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy

In [2]:
BASE_DIR = '..'
COUNTRIES_DIR = os.path.join(BASE_DIR, 'gdelt')
PROCESSED_DIR = os.path.join(BASE_DIR, 'data', 'processed')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')
CNN_TRAIN_IMAGE_DIR = os.path.join(BASE_DIR, 'data', 'cnn_images')
CNN_DIR = os.path.join(BASE_DIR, 'models', 'night28vggtrained_model.pt')

In [3]:
os.makedirs(RESULTS_DIR, exist_ok=True)
for country in ['nepal_2018']:
    os.makedirs(os.path.join(RESULTS_DIR, country), exist_ok=True)

# Feature extract with CNN
If you have run this step before, you can skip it and run the commented out code in the next section to quick-start.

In [4]:
df_images = pd.read_csv(os.path.join(PROCESSED_DIR, 'nightimage_download_actual.csv'))

In [5]:
df_images.tail()

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,consumervalue,agvalue,elevationvalue,mblphne,pop15_49,electricityvalue,pop0_5,nightlights,Noofprotest,nightlights_bin,is_train
796,26.837084_85.56125_26.837084_85.56125.png,26.837084,85.561250,26.837084,85.56125,0.649576,0.613839,0.0,0.980307,0.013164,0.965833,0.030578,0.10204,0.004601,1,True
797,26.873016611364783_85.56125_26.837084_85.56125...,26.873017,85.561250,26.837084,85.56125,0.649576,0.613839,0.0,0.980307,0.013164,0.965833,0.030578,0.10204,0.004601,1,True
798,26.80115138863522_85.59718261136479_26.837084_...,26.801151,85.597183,26.837084,85.56125,0.649576,0.613839,0.0,0.980307,0.013164,0.965833,0.030578,0.10204,0.004601,1,True
799,26.837084_85.59718261136479_26.837084_85.56125...,26.837084,85.597183,26.837084,85.56125,0.649576,0.613839,0.0,0.980307,0.013164,0.965833,0.030578,0.10204,0.004601,1,False
800,26.873016611364783_85.59718261136479_26.837084...,26.873017,85.597183,26.837084,85.56125,0.649576,0.613839,0.0,0.980307,0.013164,0.965833,0.030578,0.10204,0.004601,1,True


In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using {device} as backend')
model = torch.load(CNN_DIR, map_location=device)

Using cuda as backend


In [7]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=3, bias=True)
)

In [8]:
# rip off the final layers
model.classifier = model.classifier[:4]

In [9]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
)

In [11]:
transformer = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

# custom dataset for fast image loading and processing
# does not follow the usual style of folder -> folder for each class -> image
# we just want one folder with images
class ForwardPassDataset(torch.utils.data.Dataset):
    def __init__(self, image_dir, transformer):
        self.image_dir = image_dir
        self.image_list = os.listdir(self.image_dir)
        self.transformer = transformer

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, index):
        image_name = self.image_list[index]

        # Load image
        X = self.filename_to_im_tensor(self.image_dir + '/' + image_name)
        
        # dataloaders need to return a label, but for the forward pass we don't really care
        return X, -1
    
    def filename_to_im_tensor(self, file):
        im = plt.imread(file)[:,:,:3]
        im = self.transformer(im)
        return im

model.eval()  
classes = [0, 1, 2]
# shape of final array will be (num_validation_images, 4096)
# we also want to record the image each index represents
feats = np.zeros(((~df_images['is_train']).sum(), 4096))
image_order = []
i = 0
for c in classes:
    # use the validation images to do the forward pass
    dataset = ForwardPassDataset(os.path.join(CNN_TRAIN_IMAGE_DIR, 'valid', str(c)), transformer)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=False, num_workers=4)
    image_order += dataset.image_list
    # forward pass for this class
    for inputs, _ in tqdm(dataloader):
        inputs = inputs.to(device)
        outputs = model(inputs)
        feats[i:i+len(inputs),:] = outputs.cpu().detach().numpy()
        i += len(inputs)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [12]:
feats

array([[ 0.31067708,  0.3601594 ,  0.26862946, ..., -0.76150513,
        -0.10677798,  0.22471572],
       [ 0.54130232,  0.52983701, -0.61511183, ...,  0.17076012,
        -0.42089963, -1.17120135],
       [-0.49921608,  0.27167448, -0.09789698, ...,  0.38344195,
         0.81716973, -0.86799937],
       ...,
       [-1.17787623, -0.30852589, -0.23446774, ...,  0.4288274 ,
         0.90008843, -1.87856925],
       [-1.15110254,  0.49377331,  0.17122665, ...,  0.59835488,
         0.21176425, -1.94773054],
       [-0.10396078,  0.04080088, -0.28183877, ...,  0.91971856,
         0.53963941, -1.76287246]])

In [13]:
feats.shape

(178, 4096)

In [14]:
forward_pass_df = pd.DataFrame.from_dict({'image_name': image_order, 'feat_index': np.arange(len(image_order))})
forward_pass_df.head()

,image_name,feat_index
0,27.93448438863522_85.169586_27.970417_85.16958...,0
1,27.85634961136478_84.82218261136478_27.820417_...,1
2,27.61468261136478_85.38625_27.57875_85.38625.png,2
3,27.039682611364782_85.30031738863522_27.00375_...,3
4,27.112083_85.43625_27.112083_85.43625.png,4


In [15]:
forward_pass_df.shape

(178, 2)

In [16]:
df_protest = pd.merge(left=df_images, right=forward_pass_df, on='image_name')

In [17]:
# have we maintained all validation images?
assert len(df_protest) == (~df_images['is_train']).sum()

In [19]:
# have we maintained all validation images?
assert len(df_protest) == (~df_images['is_train']).sum()

In [20]:
df_protest.tail()

,image_name,image_lat,image_lon,cluster_lat,cluster_lon,consumervalue,agvalue,elevationvalue,mblphne,pop15_49,electricityvalue,pop0_5,nightlights,Noofprotest,nightlights_bin,is_train,feat_index
173,26.93968261136478_85.83625_26.90375_85.83625.png,26.939683,85.836250,26.903750,85.836250,0.291982,0.716957,0.029020,0.956075,0.005457,0.968001,0.008737,0.089728,0.053211,1,False,91
174,26.81781738863522_85.777916_26.85375_85.777916...,26.817817,85.777916,26.853750,85.777916,0.132762,0.737086,0.017963,0.967104,0.008040,0.884982,0.012097,0.000000,0.000000,0,False,75
175,26.81781738863522_85.81384861136479_26.85375_8...,26.817817,85.813849,26.853750,85.777916,0.132762,0.737086,0.017963,0.967104,0.008040,0.884982,0.012097,0.000000,0.000000,0,False,33
176,26.873016611364783_85.52531738863522_26.837084...,26.873017,85.525317,26.837084,85.561250,0.649576,0.613839,0.000000,0.980307,0.013164,0.965833,0.030578,0.102040,0.004601,1,False,93
177,26.837084_85.59718261136479_26.837084_85.56125...,26.837084,85.597183,26.837084,85.561250,0.649576,0.613839,0.000000,0.980307,0.013164,0.965833,0.030578,0.102040,0.004601,1,False,90


In [21]:
df_protest['Country'] = 'NP'

## Aggregate Features
For each country, we aggregate the image features per cluster and save them to results/country/cnn

In [22]:
country_abbrv = ['NP']
country_dir = ['nepal_2018']

for ca, cd in zip(country_abbrv, country_dir):
    df_c = df_protest[df_protest['Country'] == ca]
    group = df_c.groupby(['cluster_lat', 'cluster_lon'])
    x = np.zeros((len(group), 4096))
    cluster_list = [] # the corresponding clusters (lat, lon) to the x aggregate feature array
    for i, g in enumerate(group):
        lat, lon = g[0]
        im_sub = df_protest[(df_protest['cluster_lat'] == lat) & (df_protest['cluster_lon'] == lon)].reset_index(drop=True)
        agg_feats = np.zeros((len(im_sub), 4096))
        for j, d in im_sub.iterrows():
            agg_feats[j,:] = feats[d.feat_index]
        agg_feats = agg_feats.mean(axis=0) # averages the features across all images in the cluster

        x[i,:] = agg_feats
        cluster_list.append([lat, lon])
    # save to the correct directory
    save_dir = os.path.join(RESULTS_DIR, cd, 'cnn')
    os.makedirs(save_dir, exist_ok=True)
    np.save(os.path.join(save_dir, 'nighttrainvggcluster_feats.npy'), x)
    pickle.dump(cluster_list, open(os.path.join(save_dir, 'nighttrainvggcluster_order.pkl'), 'wb')) 
    

In [20]:

data_array = np.load('../results/nepal_2018/cnn/cluster_feats.npy')
data = pd.DataFrame(data_array)

In [21]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,4086,4087,4088,4089,4090,4091,4092,4093,4094,4095
0,-0.283289,0.566917,-0.118794,-0.547003,-0.474713,-0.200924,-0.297794,0.100196,-0.481585,-0.330021,...,-0.201371,-0.160594,-0.692318,-0.136503,-0.663593,0.199893,0.318315,-0.079850,0.360743,-0.577411
1,-0.391647,0.242383,0.175748,-0.184521,-0.004234,0.012669,-0.299766,0.087809,0.539612,0.181441,...,-0.224399,-0.117486,0.152750,0.252105,0.041224,0.667171,0.234246,0.257077,-0.049544,-0.844459
2,-0.616642,0.879770,-0.220837,-0.293858,-0.432047,0.068485,-0.148867,0.095733,-0.176750,-0.565605,...,0.096275,0.019890,-0.224270,-0.154993,-0.837059,0.330519,-0.033447,0.307753,0.533051,-1.320762
3,-0.373912,0.523221,-0.080454,-0.347141,-0.478730,0.176487,-0.249717,0.106497,0.335417,-0.548297,...,-0.251748,-0.090021,-0.139228,-0.091921,-0.573192,0.442339,-0.172156,0.246612,-0.009111,-0.945608
4,-0.066026,-0.045560,0.168912,-0.629117,-0.565312,-0.153071,-0.280531,0.252013,-0.364775,-0.153662,...,-0.166793,-0.078803,-0.133168,0.150548,-0.542827,0.390639,-0.650365,-0.060529,0.325392,-0.741968


In [23]:
data.shape

(4409, 4096)